## Notebook for reconstructing from a single alinged tilt-series

### 1. 3-d mask adjustments
### 2. Reconstruction
### 3. Diagnostics

In [1]:
%matplotlib qt

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pickle
from timeit import default_timer as timer
import skimage.filters as skfl

In [3]:
import hyperspy.api as hs

In [4]:
#load editable pyramid version
import sys
import os
pyramid_dev_version_path=os.path.abspath(r"D:\Programmes\Git\empyre-patched\empyre")
sys.path.append(pyramid_dev_version_path)
#print(sys.path)
import pyramid as pr
pr

<module 'pyramid' from 'D:\\Programmes\\Git\\empyre-patched\\empyre\\pyramid\\__init__.py'>

In [5]:
#Code by A.S.
import pyramidas.alignment as pa
import pyramidas.util as pu
import pyramidas.reconstruction as pre
import pyramidas.diagnostics as pd

### 3.Diagnostics

In [11]:
import matplotlib as mpl
mpl.rcParams.update({"figure.dpi":200.0, 
                     'figure.figsize': [6.0, 4.0],
                     'font.size': 12.0, 
                     'legend.fontsize': 12.0,
                     'lines.linewidth': 1})

In [7]:
with open("data_combined_refined.pickle", "rb") as f: #data_combined_refined
    data_refined=pickle.load(f)
    mask0 = data_refined.mask.copy()
    
result_folder=".//"
with open(result_folder+"reconstruction_results_em1e0_10000_mask_corrected.pickle", "rb") as f:
    results=pickle.load(f)
key=(1e-1,1e0)
magdata_rec, cost_function, cost_values, result_note = results[key]
mask_edge=5
print("Weights:",key,"Note:",result_note)

#select the voxel to evaluate
voxel_position = pd.find_voxel_coords(data_refined.mask, position_zyx=[6,40,50], mask_edge=mask_edge, plot_results=True)

#Calculate information content distributions
magdata_rec, cost_function, cost_values = results[key][:3]
mag_rec = magdata_rec
cost_f = cost_function
data = data_refined

diagnostic_results = pd.bayesian_diagnostics(data, mag_rec, cost_f, voxel_position_zyx=voxel_position, 
                       verbose=True, max_iter=1000, plot_results=True)

print("spatial resolution FWHM")
print(np.sqrt(np.average(diagnostic_results[1]**2))*10.25)

# with open(result_folder+"diagnostics_em1e0_10000_mask_corrected.pickle","wb") as f:
#     pickle.dump(diagnostic_results, f)

Weights: (0.1, 1.0) Note: 10000 iterations
finished calculating for x component
finished calculating for y component
finished calculating for z component

Voxel position: (6, 40, 50)
Magnetisation vector is:
        (M_x = 6.48e-01 +/- 9.47e-02 T,
         M_y = 4.34e-01 +/- 1.01e-01 T,
         M_z = -5.31e-02 +/- -8.64e-03 T)
Amplitude: 0.782 +/- 0.139 T
Spatial resolution (dx, dy, dz): 4.0, 3.1, 5.2 pixels
Pixel spacing: 10.25 nm
spatial resolution FWHM
42.81382543469591


In [8]:
# with open(result_folder+"diagnostics_em1e0_10000_mask_corrected.pickle","wb") as f:
#     pickle.dump(diagnostic_results, f)

### For plotting

In [7]:
result_folder=".//"
with open(result_folder+"diagnostics_em1e0_10000_mask_corrected.pickle", "rb") as f:
    diagnostic_results=pickle.load(f)
    
with open("data_combined_refined.pickle", "rb") as f: #data_combined_refined
    data_refined=pickle.load(f)
    mask0 = data_refined.mask.copy()
    

with open(result_folder+"reconstruction_results_em1e0_10000_mask_corrected.pickle", "rb") as f:
    results=pickle.load(f)
key=(1e-1,1e0)
magdata_rec, cost_function, cost_values, result_note = results[key]
mask_edge=5
print("Weights:",key,"Note:",result_note)

Weights: (0.1, 1.0) Note: 10000 iterations


#### Error from missaligned images

In [11]:
error_total, fwhm_xyz, diagnostic_vec = diagnostic_results
print(diagnostic_results)

(0.13872928105402318,
 array([3.98820157, 3.07197142, 5.19597813]),
 (<pyramid.diagnostics.Diagnostics at 0x2849a605b38>,
  <pyramid.diagnostics.Diagnostics at 0x2849a605a20>))

In [119]:
# get average FWHM in nm
print("spatial resolution FWHM")
np.sqrt(np.average(diagnostic_results[1]**2))*data_refined.a

spatial resolution FWHM


42.81006617197481

In [112]:
# average magnetisation *mu0
mag_avg=np.mean(magdata_rec.field_amp[magdata_rec.get_mask()])
print("average magnetisation")
mag_avg

average magnetisation


0.8657271679751307

In [25]:
# evaluate perturbation of forcing one phase image to be perfect
err_alignment = error_total/data_refined.count
print("relative alignment error, %, ", err_alignment/mag_avg*100)
err_alignment

relative alignment error %,  1.001537249449647


0.008670580065876449

#### Error from phase measurement noise (by measuring phase differences between reference holograms)

In [113]:
#load reference phase images and rebin to reconstruction size
raw_scaling=0.64057 #nm
rebin_factor=round(data_refined.a/raw_scaling)

paths=[]
images=[]
for i in range(1,6):
    p=f"D:\\Programmes\\Python\\pyramid\\Package testing\\bridge v3\\double series\\for diagnostics\\90phi_flip 1_+0\\ref phases\\ref_C_P frame {i}.tif"
    paths.append(p)
    img=hs.load(p).data
    img=pu.rebin_img(img, rebin_factor=rebin_factor)
    images.append(np.array(img))

#remove phase offesets and get std for each pixel

images = [im-np.mean(im) for im in images]
print("mean of each image", np.mean(images,axis=(1,2)))


std=np.std(images,axis=0)
print("corner pixel in each image",[im[0,0] for im in images])
std=np.mean(std)
print("average std per pixel, rad", std)
std

mean of each image [0.0000000e+00 0.0000000e+00 0.0000000e+00 1.9292722e-07 3.6831560e-07]
corner pixel in each image [0.3956958, 0.39381522, 0.38318023, 0.40925312, 0.3770256]
average std per pixel, rad 0.016117094


0.016117094

In [117]:
# estimage perturbation on reconstruction
diag_x=diagnostic_vec[0]
G=diag_x.gain_row
err_meas = np.sqrt(np.dot(G,G))*std
print("relative phase measurement error, %, ", err_meas/mag_avg*100)
err_meas

relative phase measurement error, %,  0.12549729958711744


0.0010864642176008172

In [116]:
# total magnitude error in T from phase perturbations
err_pert = np.hypot(err_alignment,err_meas)
print("relative phase perturbation error, %, ", err_pert/mag_avg*100)
err_pert

relative phase perturbation error, %,  1.0093693249939901


0.008738384471680164

In [190]:
data_refined.plot_mask(pretty=True, labels=False, grid=False)

In [13]:
diag_x=diagnostic_vec[0]
G=diag_x.gain_row
G

array([0., 0., 0., ..., 0., 0., 0.])

In [187]:
pos=diag_x.pos[1:]
angle_field.field[pos]

13.955624447146192

NameError: name 'pos' is not defined

In [35]:
#

### Histogram

In [10]:
#histograms
pd.histogram_magnetisation(magdata_rec[:,:,:,mask_edge:], [0,2], fit_gauss=False, save_img=False)

bin_size: 0.02 T
mean and std: 0.8531949149935405 0.37593808850896704


(array([  0.,   0.,   1.,   0.,   2.,   2.,   7.,  10.,  22.,  18.,  26.,
         37.,  53.,  54.,  75.,  89., 107., 105., 115., 151., 146., 180.,
        197., 245., 233., 248., 255., 239., 201., 207., 199., 222., 204.,
        202., 229., 221., 206., 188., 217., 223., 204., 207., 225., 199.,
        193., 194., 176., 177., 175., 180., 215., 190., 185., 148., 166.,
        129., 157., 157., 104., 122., 112., 107., 113.,  86.,  80., 101.,
         80.,  97.,  82.,  67.,  66.,  59.,  50.,  51.,  46.,  40.,  40.,
         30.,  23.,  23.,  32.,  29.,  26.,  24.,  17.,  18.,  19.,  21.,
          9.,  19.,  11.,  17.,   7.,  18.,   9.,  11.,  15.,  19.,   8.,
          8.]), 0.02)

In [49]:
def histogram_magnetisation (magdata_rec, range_hist = (1,2), n_bins=100, save_img=False, verbose=True, fit_gauss=True):
    """
    Plots a histogram of magnetisation amplitude and fits a gaussian.
    
    returns: (bin_counts, bin_edges)
    """

    amp = magdata_rec.field_amp.copy()
    amp_distribution = amp[amp>0]

    bin_size = (range_hist[1]-range_hist[0])/n_bins
    

    plt.figure(figsize=[5,4])
    bins, bin_edges, temp_patches = plt.hist(amp_distribution, bins = n_bins, range=range_hist)
    plt.xlabel("$\mu_0 M_s$, T")
    plt.ylabel("Number of voxels")
    
    #fit gaussian
    if fit_gauss:
        def gaussian_1D(x, a, b, c):
            return a* np.e**((-1/2)*((x-b)/c)**2)
        fun=gaussian_1D
        x=np.array(bin_edges[:-1])+bin_size/2 #bin centres
        y=bins
        starting_pos = (np.max(bins), np.median(x), 0.1)
        pop, pcov = op.curve_fit(fun, x, y, p0=starting_pos)
        fit_err = np.sqrt(np.diag(pcov))
        plt.plot(x, fun(x,*pop), 'r-', label="Gaussian fit")

#     plt.legend()
    plt.tight_layout()
    if save_img:
        plt.savefig("amp bin.png",dpi=200)
    plt.show()
    
    if verbose:
        print("bin_size:", bin_size, "T")
        if fit_gauss:
            print("fitting params (a,b,c) with error:", pop, fit_err)
        print("mean and std:", np.mean(amp_distribution), np.std(amp_distribution))
    
    return (bins, bin_size)

In [50]:
histogram_magnetisation (magdata_rec[:,:,:,mask_edge:], [0,2], save_img=True, fit_gauss=False)

bin_size: 0.02 T
mean and std: 0.8531949149935405 0.37593808850896704


(array([  0.,   0.,   1.,   0.,   2.,   2.,   7.,  10.,  22.,  18.,  26.,
         37.,  53.,  54.,  75.,  89., 107., 105., 115., 151., 146., 180.,
        197., 245., 233., 248., 255., 239., 201., 207., 199., 222., 204.,
        202., 229., 221., 206., 188., 217., 223., 204., 207., 225., 199.,
        193., 194., 176., 177., 175., 180., 215., 190., 185., 148., 166.,
        129., 157., 157., 104., 122., 112., 107., 113.,  86.,  80., 101.,
         80.,  97.,  82.,  67.,  66.,  59.,  50.,  51.,  46.,  40.,  40.,
         30.,  23.,  23.,  32.,  29.,  26.,  24.,  17.,  18.,  19.,  21.,
          9.,  19.,  11.,  17.,   7.,  18.,   9.,  11.,  15.,  19.,   8.,
          8.]), 0.02)

### FSC

In [45]:
#FSC
#do for all sub-components:
mag_rec = magdata_rec[:,:,:,mask_edge:]

field_array=mag_rec.field.copy()
field_array[:,:,:,:mask_edge]=0

FSCs = []
resolutions=[]
for i in range(3):
    array_3d=field_array[i,:,:,:]
    ffts = pd.fsc_split_array(array_3d)
    freq, FSC, ns_effective = pd.fsc_calculate_correlation(array_3d, *ffts, scale=3, plot_results=True)
    FSCs.append(FSC)

C:\Users\LOCAL_Aurys\HyperSpy-bundle\envs\pyramidPatched2\lib\site-packages\skimage\transform\_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
C:\Users\LOCAL_Aurys\HyperSpy-bundle\envs\pyramidPatched2\lib\site-packages\skimage\transform\_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
C:\Users\LOCAL_Aurys\HyperSpy-bundle\envs\pyramidPatched2\lib\site-packages\skimage\transform\_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting w

In [46]:
sne=np.sqrt(ns_effective)
sigma_3 = 3/(sne+2)
hbit=((0.2071+1.9102/sne)/(1.2071+0.9102/sne))

freq_p=2*np.array(freq) #for plotting convert to f/nyquist

def find_intersection(FSC, hbit, freq_p):
    for i in range(len(FSC))[10:]:
        fi=FSC[i]
        thi=[hbit[i]]
        if thi>fi:
            resolution = tick_function([freq_p[i-1]])[0]
            return(resolution)
            break
            
def tick_function(f):
    freqs=[]
    for freq in f:
        if freq<=1e-10:
            freqs.append(None)
        else:
            f=1/freq*mag_rec.a
            freqs.append("%.1f"%f)
    return (freqs) #in nm

# plt.figure()
plt.figure(figsize=[6.0, 4.4])
ax1 = plt.gca()
ax2 = ax1.twiny()

s='xyz'
fmts=["r.-","g.-","b.-"]
for i,FSC in enumerate(FSCs):
    resolution = find_intersection(FSC, hbit, freq_p)
    
    ax1.plot(freq_p, FSC, fmts[i], label=f"$m_{s[i]}$, resolution $= {resolution}$ nm")

ax1.plot(freq_p, hbit, 'k-', label="1/2 bit threshold")
#ax1.plot(freq_p, sigma_3, 'k-.', label="3 sigma")
ax1.set_xlabel("Spatial frequency / Nyquist")
ax1.legend() # loc='lower left'
ax1.set_ylabel("Correlation coefficient")


xtickslocs = ax1.get_xticks()[1:-1]*mag_rec.a/10
ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(xtickslocs)
ax2.set_xticklabels(tick_function(xtickslocs))
ax2.set_xlabel("Resolution, nm")
plt.tight_layout()
plt.savefig("FSC.png",dpi=200)


### plotting

In [121]:
#for plotting 
with open("data_combined_refined.pickle", "rb") as f:
    data_refined=pickle.load(f)
    mask0 = data_refined.mask.copy()
    
result_folder=".//"
with open(result_folder+"reconstruction_results_em1e0_10000_mask_corrected.pickle", "rb") as f:
    results=pickle.load(f)
key=(1e-1,1e-0)
magdata_rec, cost_function, cost_values, result_note = results[key]
mask_edge=5
print("Weights:",key,"Note:",result_note)

Weights: (0.1, 1.0) Note: 10000 iterations


In [122]:
angle_field = pre.inspect_magdata(magdata_rec[:,:,:,mask_edge:], plot_angles=True)


Max spin angle: 174.71739245264737


In [10]:
pre.inspect_cost_values(cost_values, print_chis=True, scale='not log')


N = 5 cost value pairs.
model, regulariser, sum
2.48153e+04; 7.76239e+02; 2.55915e+04
2.47695e+04; 7.83207e+02; 2.55527e+04
2.47583e+04; 7.89113e+02; 2.55474e+04
2.47561e+04; 7.90496e+02; 2.55466e+04
2.47553e+04; 7.91000e+02; 2.55463e+04



In [39]:
pd.histogram_magnetisation(magdata_rec[...,mask_edge:],range_hist=(0,2) )

bin_size: 0.02 T
fitting params (a,b,c) with error: [236.03521899   0.80080298   0.36850077] [7.38042736 0.01329792 0.01362243]
mean and std: 0.8220901932384945 0.31551156867126356


(array([  0.,   0.,   1.,   0.,   2.,   2.,   7.,  10.,  22.,  18.,  26.,
         37.,  53.,  54.,  75.,  89., 107., 105., 115., 151., 146., 180.,
        197., 245., 233., 248., 255., 239., 201., 207., 199., 222., 204.,
        202., 229., 221., 206., 188., 217., 223., 204., 207., 225., 199.,
        193., 194., 176., 177., 175., 180., 274., 230., 223., 160., 190.,
        163., 181., 193., 162., 163., 154., 126., 113.,  95.,  85.,  97.,
         70.,  73.,  73.,  69., 218., 130.,  48.,  18.,  12.,   3.,   1.,
          0.,   2.,   1.,   1.,   1.,   1.,   0.,   0.,   0.,   0.,   1.,
          0.,   1.,   1.,   0.,   0.,   1.,   0.,   0.,   1.,   1.,   0.,
          0.]), 0.02)

### slices

In [123]:
mag_slice=magdata_rec[...,mask_edge:]
t=mag_slice.plot_quiver3d(mode='arrow', ar_dens=2)

In [124]:
# mag_slice.field = np.sqrt(np.abs(mag_slice.field)) * np.sign(mag_slice.field)
# too_big=mag_slice.field_amp>1.25
mag_slice=mag_slice[:,:,5:-10,5:]
mag_slice.field[mag_slice.field>1] = 1
t=mag_slice.plot_quiver3d(mode='arrow', ar_dens=2, labels=False, grid=False)

In [125]:
slice_position=5
xoffset=30
yoffset=10
# navigator
mag_slice=pr.VectorData(1, magdata_rec.field[...,mask_edge:].copy())
mask = mag_slice.get_mask()
scalar = np.where(mask, 1, 0)
nav_field = pr.VectorData(1, np.array((scalar, np.zeros(mask.shape), np.zeros(mask.shape))))
nav_field.field[0,5,40,50]=4

# coloring nav
scalar[slice_position-1:slice_position+1,:,:]=scalar[slice_position-1:slice_position+1,:,:]*(-1)
#nav_field.field[:,slice_position,:,:] = nav_field.field[:,slice_position,:,:]*2
t=nav_field.plot_quiver3d(direction=(-20,-70,170) ,mode='cube', 
                          opacity=1, ar_dens=1, coloring='custom', custom_scalar=scalar, line_width=1,
                          labels=False, grid=False, orientation=False) #'arrow'

slice_mask = np.full(mask.shape, True)
slice_mask=np.array([slice_mask]*3)
slice_mask[:,slice_position:slice_position+1,:,:]=False

mag_slice.field = np.sqrt(np.abs(mag_slice.field)) * np.sign(mag_slice.field)
mag_slice.field[slice_mask]=0
mag_slice=mag_slice[:,:,:-yoffset,xoffset:]
t=mag_slice.plot_quiver3d(direction=(90,0,60), mode='arrow', ar_dens=1, coloring='angle', 
                          labels=False, orientation=False, grid=False)


In [126]:
magdata_rec.plot_quiver3d()
save_images=True
file=r"./gifs/slices"
for i in range (3,8)[::2]:

    slice_position=i
    xoffset=30
    yoffset=10
    # navigator
    mag_slice=pr.VectorData(1, magdata_rec.field[...,mask_edge:].copy())
    mask = mag_slice.get_mask()
    scalar = np.where(mask, 1, 0)
    nav_field = pr.VectorData(1, np.array((scalar, np.zeros(mask.shape), np.zeros(mask.shape))))
    nav_field.field[0,5,40,50]=4


    scalar[slice_position:slice_position+2,:,:]=scalar[slice_position:slice_position+2,:,:]*(-1)
    #nav_field.field[:,slice_position,:,:] = nav_field.field[:,slice_position,:,:]*2
    tn=nav_field.plot_quiver3d(direction=(-20,-70,200) ,mode='cube', 
                              opacity=1, ar_dens=1, coloring='custom', custom_scalar=scalar, line_width=1,
                              labels=False, grid=False, orientation=False) #'arrow'

    slice_mask = np.full(mask.shape, True)
    slice_mask=np.array([slice_mask]*3)
    slice_mask[:,slice_position,:,:]=False

    mag_slice.field = np.sqrt(np.abs(mag_slice.field)) * np.sign(mag_slice.field)
    mag_slice.field[slice_mask]=0
    mag_slice=mag_slice[:,:,:-yoffset,xoffset:]
    t=mag_slice.plot_quiver3d(direction=(90,0,60), mode='arrow', ar_dens=1, coloring='angle', 
                              labels=False, orientation=False, grid=False)
    if save_images:
        img = Image.fromarray(tn[1])
        img=img.convert("RGBA")
        fp_out=file+str(i)+"_nav.png"
        img.save(fp=fp_out, format='png', interlace=False)
        
        img = Image.fromarray(t[1])
        img=img.convert("RGBA")
        fp_out=file+str(i)+"_slice.png"
        img.save(fp=fp_out, format='png', interlace=False)

In [ ]:
mag_slice.shape

In [ ]:
mag_slice=magdata_rec[:,1:,1:,mask_edge:]
mean_field=np.mean(mag_slice.field, axis=1)
mean_field=np.pad(mean_field, ((0,0),(10,0,),(1,1)))
mag_field=np.zeros((3,1,*mean_field.shape[-2:]))
mag_field[:,0,:,:]=mean_field
mag_slice=pr.VectorData(mag_slice.a, mag_field)
mag_slice.field = np.sqrt(np.abs(mag_slice.field)) * np.sign(mag_slice.field)
print(mag_slice.field.shape)
mag_slice.plot_quiver_field(ar_dens=2, scale=0.5)

In [ ]:
np.pad?

In [ ]:
print(slice_mask.shape, mag_slice.field.shape)

In [175]:
angle_field = pre.inspect_magdata(magdata_rec[:,:,:,mask_edge:], plot_angles=True)

Max spin angle: 174.71739245264737


In [15]:
#normalise field:
angles = angle_field.field.copy()
vectors=magdata_rec[...,mask_edge:]
angles[angles>90]=90
mask = vectors.get_mask()
for i in range(3):
    vectors.field[i,mask]=vectors.field[i,mask]/vectors.field_amp[mask]*(angles[mask])
t=vectors.plot_quiver3d(mode='sphere', opacity=0.3, ar_dens=1, coloring='custom', custom_scalar=angles) #'arrow'

NameError: name 'angle_field' is not defined

In [133]:
'max spin plot'

f = np.max(angle_field.field,axis=0)
f[f>90]=90
f[f<0]=0
f_shaped=np.ones((1,*f.shape))
f_shaped[0,:,:]=f
f_shaped = np.pad(f_shaped, ((0,0),(20,0),(0,1)))
print(f_shaped.shape)
maxang = pr.ScalarData(10, f_shaped)
maxang.plot_field()

(1, 113, 65)


In [181]:
f = np.max(angle_field.field,axis=0)
f[f>90]=90
f[f<0]=0
plt.matshow(f[10:-15,10:], origin='lower', cmap='magma', vmin=0, vmax=90)
plt.axis('off')
cb=plt.colorbar(orientation='horizontal')
l=[0,30,60,90]
cb.set_ticks(l)
l=[str(d)+u'\N{DEGREE SIGN}' for d in l]
cb.set_ticklabels(l)

In [180]:
# red_ang = angle_field.field[:,0:-44,10:]
red_ang = angle_field.field.copy()[:,10:-15,10:]
red_ang[:,40:50,:]=0
plt.matshow(np.max(red_ang,axis=0))
f = np.max(red_ang,axis=1)
f[f>90]=90
f[f<0]=0
plt.matshow(f[:,:], origin='lower', cmap='magma', vmin=0, vmax=90)
plt.axis('off')
cb=plt.colorbar(orientation='horizontal')
l=[0,30,60,90]
cb.set_ticks(l)
l=[str(d)+u'\N{DEGREE SIGN}' for d in l]
cb.set_ticklabels(l)

In [ ]:
data_combined.plot_phasemaps()

In [ ]:
data_combined.plot_mask(labels=False, pretty=True)

### plots for projection maps

In [11]:
data=data_combined
masks=[]
titles=[]
folder=".//gifs"
for i in range(data.count):
    pm=data.phasemaps[i]
    pr=data.projectors[i]
    masks.append(pm.mask)
    xa=round(np.degrees(pr.tilt))
    za=round(np.degrees(pr.rotation))
    title=f"{za}.{xa}.png"
    titles.append(title)
pu.matshow_n(masks, titles)

# for i in range(data.count):
    

In [13]:
titles

['7.1.png',
 '7.21.png',
 '7.31.png',
 '7.-9.png',
 '7.-19.png',
 '7.-29.png',
 '7.-39.png',
 '7.-49.png',
 '7.-59.png',
 '-90.-4.png',
 '-90.-14.png',
 '-90.-24.png',
 '-90.-34.png',
 '-90.-44.png',
 '-90.-54.png',
 '-90.-62.png']